In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import FinanceDataReader as fdr
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import pymysql

In [2]:
nasdaq_df_raw= fdr.StockListing("NASDAQ")
nasdaq_df_raw

100%|██████████| 3758/3758 [00:05<00:00, 750.42it/s] 


,Symbol,Name,IndustryCode,Industry
0,MSFT,Microsoft Corp,57201020,소프트웨어
1,AAPL,Apple Inc,57106020,전화 및 소형 장치
2,NVDA,NVIDIA Corp,57101010,반도체
3,AMZN,Amazon.com Inc,53402010,백화점
4,META,Meta Platforms Inc,57201030,온라인 서비스
...,...,...,...,...
3753,BKHAR,Black Hawk Acquisition Rights,55601010,투자 지주 회사
3754,NEWTG,NewtekOne 8 50 Fixed Rate Senior Notes due 2029,55101050,기업 금융 서비스
3755,SIFYR,Sify Technologies Rights Exp 21st June 2024,57401010,통합 통신 서비스
3756,GRALV,Grail Inc,56202010,생명 공학 및 의학 연구


In [3]:
nasdaq_df = nasdaq_df_raw[:100]

In [ ]:
nasdaq_list = list(nasdaq_df_raw['Symbol'])
nasdaq_list

In [5]:
# 데이터의 길이가 같아야해서 MSFT의 데이터 길이랑 비교

data = yf.Ticker("MSFT")
df = data.history(period="2y")
msft_len = len(df)

In [ ]:
stock_data = pd.DataFrame(columns=["Ticker", "Date", "Industry_Sector", "Open", "High", "Low", "Volume", "Close", "Close2", "Percentage_Change"])

for idx, ticker in enumerate(nasdaq_list, start=1):
    try:
        data = yf.Ticker(ticker)
        df = data.history(period="2y")
        if len(df) != msft_len:
            print(f"{idx} / {len(nasdaq_list)}")
            continue

        df = df.reset_index()[["Date", "Open", "High", "Low", "Close", "Volume"]]
        df["Date"] = pd.to_datetime(df["Date"]).dt.strftime("%Y-%m-%d")
        df["Ticker"] = ticker
        df["Close2"] = df["Close"].shift(-1) # 한 행 씩 위로 이동
        df["Percentage_Change"] = (df["Close2"] - df["Close"]) / df["Close"] * 100 # 전날 대비 등락률 구하기

        # 산업군 추가
        sector = data.info["sector"]
        industry = data.info["industry"]
        df["Industry_Sector"] = f"{industry}/{sector}"

        # 보기 좋게 컬럼 순서 변경
        new_order = ["Ticker", "Date", "Industry_Sector", "Open", "High", "Low", "Volume", "Close", "Close2", "Percentage_Change"]
        df = df[new_order]
        df = df[:-1] # 마지막 행은 NaN이므로 삭제

        if (df["Percentage_Change"].isna().sum() > 0) or ():
            print(f"{idx} / {len(nasdaq_list)}")
            continue

        stock_data = pd.concat([stock_data, df], axis=0)
        print(f"{idx} / {len(nasdaq_list)} === {ticker}")

    except:
        print(f"{idx} / {len(nasdaq_list)}")
        continue

stock_data


In [7]:
stock_data.to_csv("2years_stock_data.csv", index=False)

In [9]:
stock_data = pd.read_csv("2years_stock_data.csv")
stock_data

,Ticker,Date,Industry_Sector,Open,High,Low,Volume,Close,Close2,Percentage_Change
0,MSFT,2022-06-14,Software - Infrastructure/Technology,239.650645,241.498199,237.341203,28651500,240.269775,247.414276,2.973533
1,MSFT,2022-06-15,Software - Infrastructure/Technology,244.023831,250.893179,242.166455,33111700,247.414276,240.741486,-2.697011
2,MSFT,2022-06-16,Software - Infrastructure/Technology,241.734046,243.149192,238.825148,33169200,240.741486,243.375214,1.094007
3,MSFT,2022-06-17,Software - Infrastructure/Technology,240.476138,246.176025,239.817705,43084800,243.375214,249.360107,2.459122
4,MSFT,2022-06-21,Software - Infrastructure/Technology,245.940166,250.352668,245.203112,29928300,249.360107,248.760651,-0.240398
...,...,...,...,...,...,...,...,...,...,...
416655,WABC,2024-06-06,Banks - Regional/Financial Services,47.500000,47.990002,47.130001,49700,47.869999,47.299999,-1.190724
416656,WABC,2024-06-07,Banks - Regional/Financial Services,47.320000,47.680000,46.840000,59300,47.299999,46.139999,-2.452431
416657,WABC,2024-06-10,Banks - Regional/Financial Services,46.810001,46.810001,45.810001,89400,46.139999,45.900002,-0.520151
416658,WABC,2024-06-11,Banks - Regional/Financial Services,45.740002,46.240002,45.480000,88900,45.900002,47.160000,2.745094


In [10]:
pivoted_stock_data = stock_data.pivot_table(index=['Industry_Sector'], columns='Date', values='Percentage_Change')

pivoted_stock_data.columns.name = None
pivoted_stock_data = pivoted_stock_data.reset_index()
pivoted_stock_data = pivoted_stock_data.ffill(axis=1)
pivoted_stock_data

,Industry_Sector,2022-06-14,2022-06-15,2022-06-16,2022-06-17,2022-06-21,2022-06-22,2022-06-23,2022-06-24,2022-06-27,...,2024-05-30,2024-05-31,2024-06-03,2024-06-04,2024-06-05,2024-06-06,2024-06-07,2024-06-10,2024-06-11,2024-06-12
0,Advertising Agencies/Communication Services,5.396453,-5.338592,2.10632,1.274958,-0.640491,1.936151,0.10523,-0.961257,-3.774886,...,-0.705163,-0.464612,0.786006,1.593199,0.565171,-1.998926,0.055454,-1.647457,2.335152,-0.310373
1,Aerospace & Defense/Industrials,3.565869,-4.431969,1.226438,0.492697,-0.183524,0.350352,2.728303,0.001512,-1.518004,...,0.544501,0.172089,-1.112981,1.548803,-1.619847,-0.173955,1.212244,0.183939,1.473461,-1.977309
2,Airlines/Industrials,2.125069,-7.654128,5.263324,-0.264405,0.896908,-1.368047,6.074691,-1.965574,-1.154899,...,2.642765,1.644125,-0.686866,1.077725,-1.040988,-0.001944,-0.876277,-1.476247,3.073602,-1.896214
3,Airports & Air Services/Industrials,1.762932,-6.634724,4.836162,1.20504,-0.390693,0.149423,3.897793,0.430816,-3.66399,...,3.465281,-12.162324,0.320061,2.03912,1.508973,-4.47301,2.789847,-4.56901,-1.17193,1.61966
4,Aluminum/Basic Materials,-0.409242,-8.531355,-2.850377,0.205712,-5.905523,-10.674147,10.907689,3.041677,-1.547113,...,-3.164624,-2.606352,-5.106204,0.338882,0.524779,-3.850682,0.51904,-2.148007,1.173044,-0.691224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,Trucking/Industrials,1.460407,-4.53985,-2.263112,2.064469,-1.986676,2.786878,3.795093,1.213375,-3.309138,...,1.804806,-1.456885,1.256353,1.808424,-1.008251,0.588225,1.40183,0.228238,0.466125,-1.550369
112,Utilities - Diversified/Utilities,1.265021,-1.62399,-1.936514,1.181615,0.479934,0.238805,3.240153,0.692308,1.084793,...,0.578104,-0.630043,-0.700784,0.974575,-1.408916,-0.731407,-1.927006,0.219605,1.487719,-1.136364
113,Utilities - Regulated Electric/Utilities,0.356432,-1.379258,-0.949574,1.220585,0.656343,1.663852,2.110506,1.158319,0.178322,...,2.471538,-0.625541,0.545296,-1.134839,-0.89066,-0.711649,0.12671,0.088452,-1.01768,0.128943
114,Utilities - Renewable/Utilities,3.649336,-5.322279,5.008123,2.358927,-2.167141,1.48898,-0.233914,0.282914,-1.450486,...,-1.530725,-1.25991,-2.866188,4.174025,-4.368418,0.509252,3.275716,0.762464,-0.819,0.121322


In [12]:
# nasdaq100 종목에만 존재하는 산업군들

nasdaq100_tickers = ['MSFT','AAPL','NVDA','GOOG','GOOGL','AMZN','META','AVGO','TSLA','ASML','COST','PEP','NFLX','AZN','AMD','LIN','ADBE','TMUS',
 'CSCO','QCOM','INTU','PDD','AMAT','TXN','CMCSA','AMGN','ISRG','INTC','HON','MU','BKNG','LRCX','VRTX','ADP','REGN','ABNB','ADI','MDLZ',
 'PANW','KLAC','SBUX','GILD','SNPS','CDNS','MELI','CRWD','PYPL','MAR','CTAS','CSX','WDAY','NXPI','ORLY','CEG','PCAR','MNST','MRVL','ROP',
 'CPRT','DASH','DXCM','FTNT','MCHP','AEP','KDP','ADSK','TEAM','LULU','KHC','PAYX','ROST','MRNA','DDOG','TTD','ODFL','FAST','IDXX','EXC',
 'CHTR','CSGP','GEHC','FANG','EA','VRSK','CCEP','CTSH','BKR','BIIB','XEL','ON','CDW','ANSS','MDB','DLTR','ZS','GFS','TTWO','ILMN','WBD',
 'WBA','SIRI']

nasdaq100_industries = []
for ticker in nasdaq100_tickers:
    data = yf.Ticker(ticker)
    industry = f"{data.info['industry']}/{data.info['sector']}"
    nasdaq100_industries.append(industry)
nasdaq100_industries = list(set(nasdaq100_industries))

In [13]:
len(nasdaq100_industries)

44

In [16]:
## AWS RDS 정보
HOST = "nasdaq-db.clcgk04eeism.ap-northeast-2.rds.amazonaws.com"


In [ ]:
USER = "hoo"
PASSWORD = "hoo1234"
DB = "nasdaq"
CHARSET = "utf8"

In [17]:
# MySQL 함수 선언
SQL1 = '''
CREATE TABLE network (
    dailydate varchar(20),
    center varchar(255),
    ind1 varchar(255),
    corr1 float,
    ind2 varchar(255),
    corr2 float,
    ind3 varchar(255),
    corr3 float,
    ind4 varchar(255),
    corr4 float,
    ind5 varchar(255),
    corr5 float,
    PRIMARY KEY (center, dailydate)
);
'''

SQL2 = '''
INSERT INTO network (dailydate, center, ind1, corr1, ind2, corr2, ind3, corr3, ind4, corr4, ind5, corr5) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
'''

def mysql_connection(host, user, password, db, charset):
    return pymysql.connect(host=host, user=user, password=password, db=db,charset=charset)

def insert_data(connection, SQL, *args):
    cursor = connection.cursor(pymysql.cursors.DictCursor)
    cursor.execute(SQL, args)
    connection.commit()
    
def create_table(connection, SQL):
    cursor = connection.cursor(pymysql.cursors.DictCursor)
    cursor.execute(SQL)
    connection.commit()

# MySQL connection 생성
connection = mysql_connection(HOST, USER, PASSWORD, DB, CHARSET)

In [ ]:
# create_table(connection, SQL1)

In [ ]:
df = pd.DataFrame(pivoted_stock_data)

# 'Industry_Sector' 열을 인덱스로 설정
df.set_index('Industry_Sector', inplace=True)

# 데이터프레임을 딕셔너리로 변환 (섹터명을 키로 사용)
sector_dict = df.T.to_dict()

# 결과 출력
for sector, returns in sector_dict.items():
    print(f"{sector}: {returns}")

In [ ]:
today = datetime.now().strftime("%Y-%m-%d")

for n100_industry in nasdaq100_industries:
    center_industry = n100_industry
    center_industry_returns = df.loc[center_industry]

    correlations = {}
    for industry in df.index:
        if industry != center_industry:
            industry_return = df.loc[industry]
            combined_df = pd.concat([center_industry_returns, industry_return], axis=1)
            combined_df.columns = ['Center_Industry', 'Industry_Return']
            correlation = combined_df['Center_Industry'].corr(combined_df['Industry_Return'])
            correlations[industry] = correlation
    top_correlations = {k: correlations[k] for k in sorted(correlations, key=correlations.get, reverse=True)[:5]}
    keys = list(top_correlations.keys())
    values = list(top_correlations.values())
    
    insert_data(connection, SQL2, today, n100_industry, keys[0], values[0], keys[1], values[1], keys[2], values[2], keys[3], values[3], keys[4], values[4])
    print(top_correlations)
